In [1]:
!pip install opencv-python

In [2]:
import numpy as np
import time
from faker import Faker
import pandas as pd
import numpy as np
import cv2
import algoritm_moves



TILE_SIZE = 32

MARKET = """
##################
##..............##
##..##..##..#e..B#
##..##..##..#s..o#
##..##..##..#g..a#
##..##..##..#m..c#
##..##..##..#b..p#
##..............##
##..C#..C#..C#..##
##..##..##..##..##
##..............##
##############GG##
""".strip()

transition_matrix = np.array([  #dairy       #spices     #drinks     #fuits     #checkout   #entry
#dairy                          
                                [0.73733108, 0.05140766, 0.05861486, 0.04988739, 0.10275901,0],
#spices                         
                                [0.1933684 , 0.40235932, 0.16323928, 0.09102503, 0.15000797,0],
#drinks                         
                                [0.01090086, 0.08700123, 0.59862197, 0.08792678, 0.21554916,0],
#fuits                          
                                [0.09597669, 0.05070467, 0.05487757, 0.59727581, 0.20116526,0],
#checkout                       
                                [0.        , 0.        , 0.        , 0.        , 1.        ,0],
#entry                          
                                [0.287441  , 0.181330  , 0.153392  , 0.377300  , 0.000537  ,0]
])


entry_probs = {
            0: 0.4006741573033708,
            1: 0.16157303370786516,
            2: 0.14449438202247192,
            3: 0.12,
            4: 0.07460674157303371,
            5: 0.05393258426966292,
            6: 0.027640449438202246,
            7: 0.010786516853932584,
            8: 0.004044943820224719,
            9: 0.0020224719101123597,
            11: 0.00022471910112359551}

In [3]:
class SupermarketMap:
    """Visualizes the supermarket background"""

    def __init__(self, layout, tiles):
        """
        layout : a string with each character representing a tile
        tiles   : a numpy array containing all the tile images
        """
        self.tiles = tiles
        # split the layout string into a two dimensional matrix
        self.contents = [list(row) for row in layout.split("\n")]
        self.ncols = len(self.contents[0])
        self.nrows = len(self.contents)
        self.image = np.zeros(
            (self.nrows*TILE_SIZE, self.ncols*TILE_SIZE, 3), dtype=np.uint8
        )
        self.prepare_map()

    def extract_tile(self, row, col):
        """extract a tile array from the tiles image"""
        y = row*TILE_SIZE
        x = col*TILE_SIZE
        return self.tiles[y:y+TILE_SIZE, x:x+TILE_SIZE]

    def get_tile(self, char):
        """returns the array for a given tile character"""
        if char == "#":
            return self.extract_tile(0, 0)
        elif char == "G":
            return self.extract_tile(7, 3)
        elif char == "C":
            return self.extract_tile(2, 8)

### adding fruits
        elif char == "b":
            return self.extract_tile(0, 4)
        elif char == "m":
            return self.extract_tile(3, 4)
        elif char == "g":
            return self.extract_tile(4, 4)
        elif char == "p":
            return self.extract_tile(5, 4)
        elif char == "c":
            return self.extract_tile(7, 4)
        elif char == "o":
            return self.extract_tile(3, 6)
        elif char == "s":
            return self.extract_tile(1, 5)
        elif char == "B":
            return self.extract_tile(7, 10)
        elif char == "e":
            return self.extract_tile(1, 11)
        elif char == "a":
            return self.extract_tile(1, 12)


        else:
            return self.extract_tile(1, 2)

    def prepare_map(self):
        """prepares the entire image as a big numpy array"""
        for row, line in enumerate(self.contents):
            for col, char in enumerate(line):
                bm = self.get_tile(char)
                y = row*TILE_SIZE
                x = col*TILE_SIZE
                self.image[y:y+TILE_SIZE, x:x+TILE_SIZE] = bm

    def draw(self, frame):
        """
        draws the image into a frame
        """
        frame[0:self.image.shape[0], 0:self.image.shape[1]] = self.image

    def write_image(self, filename):
        """writes the image into a file"""
        cv2.imwrite(filename, self.image)

In [4]:
class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """
    def __init__(self, name, state, transition_probs, supermarket, avatar, row, col):
        self.name = name
        self.state = state
        self.transition_probs = transition_probs
        self.supermarket = supermarket
        self.avatar = avatar
        self.row = row
        self.col = col

    def __repr__(self):
        return f'{self.name} is in {self.state}'

    def next_state(self):
        '''
        Propagates the customer to the next state.
        Returns nothing.
        '''
        state_dict = {'dairy':0, 'spices':1, 'drinks':2, 'fruit':3, 'checkout':4, 'entry':5} 
        self.state = np.random.choice(
            ['dairy', 'spices', 'drinks', 'fruit', 'checkout', 'entry'], 
            p=self.transition_probs[state_dict[f'{self.state}']]
            )
    
    def is_active(self):
        """
        Returns True if the customer has not reached the checkout yet.
        """
        active = self.state != 'checkout'
        return active

    def draw(self, frame):
        x = self.col * TILE_SIZE
        y = self.row * TILE_SIZE
        print(frame[y:y+TILE_SIZE, x:x+TILE_SIZE].shape,self.avatar.shape)
        frame[y:y+TILE_SIZE, x:x+TILE_SIZE] = self.avatar # simple creation of the avatar
        if x > 0 & x < 13:
            if y > 0 & y < 19: ### check if coordinates are on the map
                frame[y:y+TILE_SIZE, x:x+TILE_SIZE] = self.avatar
            else:
                cv2.destroyWindow('customer')


        self.next_position()

    def next_position(self):
        """propagates all customers to the next state.
        """
        self.row = self.row -1
        self.col = self.col
        return None
#    def __repr__(self):
#        return f"Customer {self.id}, is in {self.row} / {self.col}."
   


In [5]:
class Supermarket:
    """
    Manages multiple Customer instances that are currently in the market.
    """

    def __init__(self,market_map):
        self.customers = {}
        self.minutes = 0
        self.last_id = 0
        self.market_map = market_map

    def __repr__(self):
        return f'Supermarket customers: {len(self.customers)}, \
                  Time: {self.minutes}, Last customer ID: {self.last_id}'

    def get_time(self):
        """
        Current time in HH:MM format,
        """
        return self.minutes 

    def add_new_customers(self):
        """
        Randomly creates new customers.
        """
        entry_no = np.random.choice(list(entry_probs.keys()),p = list(entry_probs.values()))       
        avatar = tiles[8*32:9*TILE_SIZE, 2*32:3*TILE_SIZE]
        entry_no =1 # TODO this is one just or testing!!!
        entry_position = (11,15)
        for _ in range(entry_no):
            self.last_id +=1
            entry_point = 'entry'
            f = Faker()
            name = f.name()
            customer = Customer(name, entry_point, transition_matrix, self.market_map, avatar, entry_position[0],entry_position[1])
            self.customers.update({self.last_id : customer})
            print(f'{customer.name} entered the supermarket')
        return None

    def move_customers(self):
        """
        Move all customers randomly.
        """
        for _,customer in self.customers.items():
            customer.next_state()
        return None

    def print_customers(self):
        """
        Print all customers with the current time and id in CSV format.
        """
        for key,customer in list(self.customers.items()):
            print(customer)
        return None

    def remove_exitsting_customers(self):
        """
        Removes every customer that is not active any more.
        """
        for key,customer in list(self.customers.items()):
            if not customer.is_active():
                del self.customers[key]
        return None

    def next_minute(self):
        """
        Propagates all customers to the next state.
        """
        self.minutes +=1
        self.remove_exitsting_customers()
        self.add_new_customers()
        self.move_customers()        
        self.print_customers()
        print(f'--> Minutes: {self.minutes}, last ID: {self.last_id}, customers inside: {len(self.customers)}')

    


In [6]:
background = np.zeros((384, 576, 3), np.uint8)
tiles = cv2.imread(
    "./graphic_simulation/tiles.png"
    )
market = SupermarketMap(MARKET, tiles)
DOODL = Supermarket(market)

In [7]:
DOODL.add_new_customers()
#DOODL.next_minute()

Roger Shaw entered the supermarket


In [8]:
type(DOODL.customers[1])

__main__.Customer

In [9]:
if __name__ == "__main__":
    while True:
        frame = background.copy()
        market.draw(frame)
        DOODL.customers[1].draw(frame)

        # https://www.ascii-code.com/
        key = cv2.waitKey(100)
       
        if key == 113: # 'q' key
            break
    
        cv2.imshow("frame", frame)

        time.sleep(10)
        cv2.destroyAllWindows()

    market.write_image("supermarket.png")

(32, 32, 3) (32, 32, 3)
(32, 32, 3) (32, 32, 3)
(32, 32, 3) (32, 32, 3)
(32, 32, 3) (32, 32, 3)
(32, 32, 3) (32, 32, 3)
(32, 32, 3) (32, 32, 3)


KeyboardInterrupt: 

In [ ]:
cv2.destroyAllWindows()